# pdum.gcp Interactive Tour

> A hands-on walkthrough of the public API for the `pdum.gcp` package.

Important:
- This notebook contains both read-only and mutation examples.
- Only run mutation cells at the end and only if you understand the consequences.
- You must have Application Default Credentials (ADC) configured (e.g., `gcloud auth application-default login`).

Mutation Disclaimer:
- Enabling APIs or creating projects may incur costs or change production state.
- The final section includes a simple `RUN_MUTATIONS` guard for safety. Set it to `True` only when you're ready.

In [1]:
# Core imports
from pdum.gcp import (
    __version__,
    get_email,
    get_iam_policy,
    list_organizations,
    lookup_api,
    quota_project,
    walk_projects,
    NO_ORG,
    NO_BILLING_ACCOUNT,
    Organization,
    Folder,
    Project,
)
print("pdum.gcp version:", __version__)

pdum.gcp version: 0.1.0-alpha


## Identity
Use the currently active ADC to discover the acting identity (email).

In [2]:
email = get_email()
print("Active identity:", email)

Active identity: nehal.alum@gmail.com


## Organizations
List organizations accessible to this identity. If you have standalone projects (no org), `NO_ORG` will be included.

In [3]:
orgs = list_organizations()
print("Found", len(orgs), "container(s)")
for o in orgs:
    print(f"- {o.display_name} ({o.resource_name})")

Found 2 container(s)
- grtig-horizons.com (organizations/18100153825)
- No Organization (NO_ORG)


Pick an organization to explore. If you only have `NO_ORG`, some steps will be skipped or use `NO_ORG` paths.

In [4]:
org = next((o for o in orgs if isinstance(o, Organization)), NO_ORG)
print("Selected container:", org.display_name, org.resource_name)

Selected container: grtig-horizons.com organizations/18100153825


## Exploring the container
Explore folders and projects; `tree()` prints a visual hierarchy (read-only).

In [5]:
# Folders (direct children)
try:
    folders = org.folders()
    print("Folders:", [f.display_name for f in folders])
except Exception as e:
    print("No folders or error listing folders:", e)

# Projects (direct children)
try:
    projs = org.projects()
    print("Projects:", [p.id for p in projs])
except Exception as e:
    print("No projects or error listing projects:", e)

# Visual tree (read-only pretty print)
org.tree()

Folders: ['Admin', 'Test']
Projects: ['admin-74726']
🌺 grtig-horizons.com (organizations/18100153825)
├── 🎸 Admin (folders/134627768985)
├── 🎸 Test (folders/673571045739)
└── 🎵 admin-74726 ()


## Navigation via cd()
Navigate folders by path (if you have folders).

In [6]:
# Example: adjust the path to a real one in your org
try:
    maybe_folder = org.cd("Test")
    print("Navigated to:", maybe_folder.display_name, maybe_folder.resource_name)
except Exception as e:
    print("cd() example skipped or failed:", e)

Navigated to: Test folders/673571045739


## Projects without Organization
If you have personal/standalone projects, they appear under `NO_ORG`.

In [7]:
no_org_projects = NO_ORG.projects()
print("NO_ORG projects:", [p.id for p in no_org_projects])

NO_ORG projects: ['h-papadum-admin-9b1b3b9b', 'oxygen-409914', 'dogwood-theorem-316813', 'texttospeech-263822']


## Quota Project & Enabled APIs
Find your quota project from ADC, and list its enabled APIs (read-only).

In [8]:
qp = quota_project()
print("Quota project:", qp.id, qp.lifecycle_state)
enabled = qp.enabled_apis()
print("Enabled APIs (first 10):", enabled[:10])

Quota project: dogwood-theorem-316813 ACTIVE
Enabled APIs (first 10): ['aiplatform.googleapis.com', 'artifactregistry.googleapis.com', 'bigquery.googleapis.com', 'bigquerystorage.googleapis.com', 'cloudapis.googleapis.com', 'cloudbilling.googleapis.com', 'cloudresourcemanager.googleapis.com', 'cloudtrace.googleapis.com', 'compute.googleapis.com', 'dataflow.googleapis.com']


## IAM Policy & Roles
Use the top-level helper to fetch IAM policy for any Resource, and list roles bound to the current user.

In [9]:
# Policy for the selected container
policy = get_iam_policy(org)
print("Bindings:", len(policy.get('bindings', [])))

# Roles for current user on container and on quota project
container_roles = org.list_roles()
project_roles = qp.list_roles()
print("Container roles:", [r.name for r in container_roles])
print("Project roles:", [r.name for r in project_roles])

Bindings: 7
Container roles: ['roles/billing.admin', 'roles/iam.securityAdmin', 'roles/orgpolicy.policyAdmin', 'roles/resourcemanager.folderAdmin', 'roles/resourcemanager.organizationAdmin']
Project roles: ['roles/owner']


In [10]:
policy

{'version': 1,
 'etag': 'BwZCafPLojY=',
 'bindings': [{'role': 'roles/billing.admin',
   'members': ['user:nehal.alum@gmail.com', 'user:nehal@grtig-horizons.com']},
  {'role': 'roles/billing.creator', 'members': ['domain:grtig-horizons.com']},
  {'role': 'roles/iam.securityAdmin',
   'members': ['user:nehal.alum@gmail.com', 'user:nehal@grtig-horizons.com']},
  {'role': 'roles/orgpolicy.policyAdmin',
   'members': ['user:nehal.alum@gmail.com', 'user:nehal@grtig-horizons.com']},
  {'role': 'roles/resourcemanager.folderAdmin',
   'members': ['user:nehal.alum@gmail.com', 'user:nehal@grtig-horizons.com']},
  {'role': 'roles/resourcemanager.organizationAdmin',
   'members': ['user:nehal.alum@gmail.com', 'user:nehal@grtig-horizons.com']},
  {'role': 'roles/resourcemanager.projectCreator',
   'members': ['domain:grtig-horizons.com']}]}

## API Lookup
Map human-friendly API names to service IDs.

In [11]:
print(lookup_api("Compute Engine API"))
print(lookup_api("Big Query"))

compute.googleapis.com
bigquery.googleapis.com


# Billing Accounts

In [12]:
org.billing_accounts()

[BillingAccount(id='014DCD-0271A5-2E6B99', display_name='ISC', status='OPEN')]

In [13]:
org.billing_accounts(open_only=False)

[BillingAccount(id='010133-839A4B-E2C512', display_name='My Billing Account', status='CLOSED'),
 BillingAccount(id='014DCD-0271A5-2E6B99', display_name='ISC', status='OPEN')]

In [14]:
NO_ORG.billing_accounts(open_only=False)

[BillingAccount(id='010133-839A4B-E2C512', display_name='My Billing Account', status='CLOSED'),
 BillingAccount(id='014DCD-0271A5-2E6B99', display_name='ISC', status='OPEN'),
 BillingAccount(id='016EF2-C9E743-5BA3D3', display_name='ISC', status='OPEN'),
 BillingAccount(id='017525-5631A5-F1195C', display_name='My Billing Account', status='CLOSED'),
 BillingAccount(id='01DA2C-8306EE-8C3D84', display_name='Nehal', status='OPEN')]

In [15]:
bill_api = lookup_api("Cloud Billing API")
print("Cloud Billing API ID:", bill_api)

Cloud Billing API ID: cloudbilling.googleapis.com


---
# Mutations (Run Carefully, Optional)
The following cells mutate GCP state. Review carefully before running.

In [ ]:
RUN_MUTATIONS = False  # set to True only if you understand the consequences
print("RUN_MUTATIONS =", RUN_MUTATIONS)

## Enable APIs for a Project
Batch-enable a set of APIs on the quota project (or another project).

In [ ]:
apis_to_enable = [
    lookup_api("Compute Engine API"),
    lookup_api("Cloud Resource Manager API"),
]
if RUN_MUTATIONS:
    result = qp.enable_apis(apis_to_enable, verbose=True, timeout=300)
    print("Enable APIs operation done:", result.get("done"))
else:
    print("Skipping API enablement mutation (set RUN_MUTATIONS=True to run)")

## Create a Project
Create a new project under the selected container (`org` or `NO_ORG`). Optionally attach billing.

In [ ]:
new_project_id = Project.suggest_name(prefix="demo")  # or pick your own
new_display_name = "Demo Project"
# Provide a billing account id to attach, or use NO_BILLING_ACCOUNT to skip
billing = NO_BILLING_ACCOUNT  # or: BillingAccount(id="012345-567890-ABCDEF", display_name="My Billing")

if RUN_MUTATIONS:
    created = org.create_project(
        new_project_id,
        new_display_name,
        billing_account=billing,
        timeout=600,
        polling_interval=5,
    )
    print("Created project:", created.id, created.lifecycle_state)
else:
    print("Skipping project creation (set RUN_MUTATIONS=True to run)")